In [71]:
from pdfminer.high_level import extract_text
from tqdm import tqdm
import os, re
from collections import OrderedDict

raw_data_dir = './raw_data'
save_dir = './data'

# straight-forward way to bypass some edge cases that question does not contain question mark (?!?)
bypass_questions = {
  'Hoa_hoc': ['7. Về không khí'],
  'Dong_vat': ['213. Tại sao lạc đà được gọi là "chiếc thuyền của sa mạc"'],
  'Khoa_hoc_moi_truong' : ['108. Vì sao bờ biển, rừng núi hoặc nông thôn không khí đặc biệt tươi mát'],
  'Toan_hoc': ['9. Cách tính nhanh các tích số của các con số gần với 10..., 100..., 1000...']
}
skipped_idx = {
    'Toan_hoc': [30, 57, 58, 59, 60, 79, 80, 120] + list(range(134, 141, 1)) + list(range(154, 161, 1)) + [178, 179, 180, 200]
}

In [76]:
def process_pdf_content(raw_text, topic, save_dir):
    content = re.sub('\s+', ' ', raw_text)
    content = re.split('(Table of Contents)\s+(LỜI NHÀ XUẤT BẢN)', content)[0]
    content = content.split('Đường nguyên còn gọi là đường glucogen – sinh thành từ đường glucoza mất nước – là một loại hidratcacbon quan trọng cung cấp năng lượng cho cơ thể.')[0]
    
    # extract questions manually
    iter = re.finditer('\s[0-9]+\.?\s', content)
    questions = list()
    incremental_idx = 1
    for m in iter:
        idx = m.start()
        question = content[idx:].strip()
        # idk what else to do, sometimes question will contain "1 " at the beginning
        if question[:2] == '1 ':
            question = question[2:]
        if question.split()[0].replace('.', '') != str(incremental_idx):
            continue
        question = question.split('?', 1)[0]
        try:
            bypass_q = [bypass_q for bypass_q in bypass_questions.get(topic, []) if bypass_q in question][0]
        except:
            bypass_q = None
        if len(question) > 150 and bypass_q == None:
            continue
        questions.append(bypass_q if bypass_q else question)
        incremental_idx = incremental_idx + 1
        if incremental_idx in skipped_idx.get(topic, []):
            incremental_idx = incremental_idx + 1
    
    print(questions)

    # split content by question and save result
    QA_map = OrderedDict()
    incremental_idx = 1
    for question in questions:
        try:
            # handle exception when question parsed incorrectly
            answer = content.split(question)[1]
            QA_map[question] = answer
            incremental_idx = incremental_idx + 1
        except:
            pass
 
    print(f'{len(QA_map)} questions in total')
    questions = list(QA_map)
    for i in tqdm(range(len(questions))):
        question = questions[i]
        answer = QA_map[question]
        if i < len(questions) - 1:
            answer = answer.split(questions[i+1])[0]
        answer = answer.replace('T ừ', 'Từ')
        with open(f'{save_dir}/{topic}/Q{i+1}.txt', 'w') as f:
            f.write(re.sub('^[0-9]+\.?\s', '', question) + "?")
        with open(f'{save_dir}/{topic}/A{i+1}.txt', 'w') as f:
            f.write(re.sub('^[\s\?]+', '', answer).strip())

In [55]:
def extract_pdf(file_path, save_dir):
    print(f'Processing {file_path}')
    raw_text = extract_text(file_path)
    process_pdf_content(raw_text, file_path.split('/')[-1].split('.pdf')[0], save_dir)

In [77]:
list_pdfs = os.listdir(raw_data_dir)
processed = ['Trai_dat.txt', 'Hoa_hoc.txt', 'Dong_vat.txt', 'Khoa_hoc_moi_truong.txt']
#for pdf in list_pdfs:
#    if pdf.endswith('.txt') and not pdf in processed:
with open(f'./raw_data/Toan_hoc.txt', 'r') as f:
    print(f'Processing Toan_hoc.txt')
    raw_text = '\n'.join(f.readlines())
    process_pdf_content(raw_text, pdf.split('/')[-1].split('.txt')[0], save_dir)

Processing Toan_hoc.txt


100%|██████████| 10/10 [00:00<00:00, 475.20it/s]

['1. Phải chăng số 0 chỉ có nghĩa là không có', '2 lá, chập 7 dư 1 lá, vậy có bao nhiêu lá cờ', '3 thì dư 2, chia 5 thì dư 3', '4 + 8 + 16 + 32 + 64 + 128 = 255. Thế dãy số trên đây từ đâu mà có', '5 kΩ và 7,5 kΩ, tính điện trở tương đương của mạch đó', '6 sẽ có diện tích bằng nửa diện tích của khu đất đãchọn. 112. Bài toán chia đất', '7 47 10 42 8 48 9 5 46 12 10 49 7 12 50 5 Để cửa hàng nhận được lợi nhuận tối đa, cần phải nhập bao nhiêu hàng', '8 48 9 5 46 12 10 49 7 12 50 5 Để cửa hàng nhận được lợi nhuận tối đa, cần phải nhập bao nhiêu hàng', '9 5 46 12 10 49 7 12 50 5 Để cửa hàng nhận được lợi nhuận tối đa, cần phải nhập bao nhiêu hàng', '10 49 7 12 50 5 Để cửa hàng nhận được lợi nhuận tối đa, cần phải nhập bao nhiêu hàng']
10 questions in total
